In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.1 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.43.1
    Uninstalling bitsandbytes-0.43.1:
      Successfully uninstalled bitsandbytes-0.43.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# #Unzip test images folder
# !unzip /content/drive/MyDrive/MeeshoAIChallenge/test_images.zip -d /content/drive/MyDrive/MeeshoAIChallenge/

In [ ]:
import pandas as pd
import os
from PIL import Image
import csv

df = pd.read_parquet('/content/drive/MyDrive/MeeshoAIChallenge/category_attributes.parquet')
df

,Category,No_of_attribute,Attribute_list
0,Men Tshirts,5,"[color, neck, pattern, print_or_pattern_type, ..."
1,Sarees,10,"[blouse_pattern, border, border_width, color, ..."
2,Kurtis,9,"[color, fit_shape, length, occasion, ornamenta..."
3,Women Tshirts,8,"[color, fit_shape, length, pattern, print_or_p..."
4,Women Tops & Tunics,10,"[color, fit_shape, length, neck_collar, ocassi..."


In [ ]:
attr_mens_tshirt = list(df["Attribute_list"][0])
attr_sarees = list(df["Attribute_list"][1])
attr_kurtis = list(df["Attribute_list"][2])
attr_womens_tshirts = list(df["Attribute_list"][3])
attr_womens_tops_tunics = list(df["Attribute_list"][4])
attr_womens_tops_tunics[4] = "occasion"

In [ ]:
men_tshirts_df = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive/Men_Tshirts.csv")
sarees_df = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive/Sarees.csv")
kurtis_df = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive/Kurtis.csv")
womens_tshirts_df = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive/Women_Tshirts.csv")
womens_tops_tunics_df = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive/Women_Tops_Tunics.csv")

sub_attr_men_tshirts = {attr: men_tshirts_df[attr].dropna().unique().tolist() for attr in attr_mens_tshirt}
sub_attr_sarees = {attr: sarees_df[attr].dropna().unique().tolist() for attr in attr_sarees}
sub_attr_kurtis = {attr: kurtis_df[attr].dropna().unique().tolist() for attr in attr_kurtis}
sub_attr_womens_tshirts = {attr: womens_tshirts_df[attr].dropna().unique().tolist() for attr in attr_womens_tshirts}
sub_attr_womens_tops_tunics = {attr: womens_tops_tunics_df[attr].dropna().unique().tolist() for attr in attr_womens_tops_tunics}


In [ ]:
df_test_mens_tshirts = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive2/test_Men_Tshirts.csv")
df_test_sarees = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive2/test_Sarees.csv")
df_test_kurtis = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive2/test_Kurtis.csv")
df_test_womens_tshirts = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive2/test_Women_Tshirts.csv")
df_test_womens_tops_tunics = pd.read_csv("/content/drive/MyDrive/MeeshoAIChallenge/archive2/test_Women_Tops_Tunics.csv")

In [ ]:
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5-int4', trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained('openbmb/MiniCPM-Llama3-V-2_5-int4', trust_remote_code=True)
model.eval()

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

MiniCPMV(
  (llm): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
            (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      

In [ ]:
def generate_combined_question(image, attributes):
    question_parts = []
    for attr, options in attributes.items():
        options_text = ", ".join(options)
        question_parts.append(f"{attr} ({options_text})")

    question = (
        "Please describe the following attributes of this clothing item using exactly this format for each attribute -\n"
        "'Attribute: Attribute_Response'\n\n"
        "Attributes to describe: " + "; ".join(question_parts) + "."
    )
    question += "\nImportant: You must provide an answer for ALL attributes listed above, without missing any."

    system_prompt = (
        "You are an assistant helping to identify attributes of clothing items. "
        "You must follow these strict rules:\n"
        "1. Respond using ONLY this format for each attribute: 'Attribute: Attribute_Response'\n"
        "2. Provide answers for ALL attributes mentioned in the question\n"
        "3. Use only single-word responses from the provided options for each attribute\n"
        "4. Do not add any explanations, greetings, or additional text\n"

        "Example format:\n"
        "Color: Blue\n"
        "Pattern: Solid\n"
        "Style: Casual"
    )

    msgs = [{'role': 'user', 'content': question}]
    res = model.chat(
        image=image,
        msgs=msgs,
        tokenizer=tokenizer,
        sampling=True,
        temperature=0.5,
        system_prompt=system_prompt
    )
    return res

In [ ]:
csv_file_path = "/content/drive/MyDrive/MeeshoAIChallenge/output_mens_tshirts.csv"
category = "Men Tshirts"
length = 5

if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["id", "Category", "len", "response"])

for row in df_test_mens_tshirts.itertuples(index=False):
    image_id = str(row.id).zfill(6)
    image_path = f"/content/drive/MyDrive/MeeshoAIChallenge/test_images/{image_id}.jpg"
    print(image_path)

    if os.path.exists(image_path):
        raw_image = Image.open(image_path).convert('RGB')
        predicted_attributes = generate_combined_question(raw_image, sub_attr_men_tshirts)

        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
            writer.writerow([image_id, category, length, predicted_attributes])
    else:
        print(f"Image {image_id}.jpg not found.")

/content/drive/MyDrive/MeeshoAIChallenge/test_images/000000.jpg
Image 000000.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000001.jpg
Image 000001.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000002.jpg
Image 000002.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000003.jpg
Image 000003.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000004.jpg
Image 000004.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000005.jpg
Image 000005.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000006.jpg
Image 000006.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000007.jpg
Image 000007.jpg not found.
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000008.jpg
/content/drive/MyDrive/MeeshoAIChallenge/test_images/000009.jpg


In [ ]:
csv_file_path = "/content/drive/MyDrive/MeeshoAIChallenge/output_sarees.csv"
category = "Sarees"
length = 10

if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["id", "Category", "len", "response"])

for row in df_test_sarees.itertuples(index=False):
    image_id = str(row.id).zfill(6)
    image_path = f"/content/drive/MyDrive/MeeshoAIChallenge/test_images/{image_id}.jpg"
    print(image_path)

    if os.path.exists(image_path):
        raw_image = Image.open(image_path).convert('RGB')
        predicted_attributes = generate_combined_question(raw_image, sub_attr_sarees)

        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
            writer.writerow([image_id, category, length, predicted_attributes])
    else:
        print(f"Image {image_id}.jpg not found.")

In [ ]:
csv_file_path = "/content/drive/MyDrive/MeeshoAIChallenge/output_kurtis.csv"
category = "Kurtis"
length = 9

if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["id", "Category", "len", "response"])

for row in df_test_kurtis.itertuples(index=False):
    image_id = str(row.id).zfill(6)
    image_path = f"/content/drive/MyDrive/MeeshoAIChallenge/test_images/{image_id}.jpg"
    print(image_path)

    if os.path.exists(image_path):
        raw_image = Image.open(image_path).convert('RGB')
        predicted_attributes = generate_combined_question(raw_image, sub_attr_kurtis)

        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
            writer.writerow([image_id, category, length, predicted_attributes])
    else:
        print(f"Image {image_id}.jpg not found.")

In [ ]:
csv_file_path = "/content/drive/MyDrive/MeeshoAIChallenge/output_womens_tshirts.csv"
category = "Women Tshirts"
length = 8

if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["id", "Category", "len", "response"])

for row in df_test_womens_tshirts.itertuples(index=False):
    image_id = str(row.id).zfill(6)
    image_path = f"/content/drive/MyDrive/MeeshoAIChallenge/test_images/{image_id}.jpg"
    print(image_path)

    if os.path.exists(image_path):
        raw_image = Image.open(image_path).convert('RGB')
        predicted_attributes = generate_combined_question(raw_image, sub_attr_womens_tshirts)

        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
            writer.writerow([image_id, category, length, predicted_attributes])
    else:
        print(f"Image {image_id}.jpg not found.")

In [ ]:
csv_file_path = "/content/drive/MyDrive/MeeshoAIChallenge/output_womens_tops_tunics.csv"
category = "Women Tops & Tunics"
length = 10

if not os.path.exists(csv_file_path):
    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["id", "Category", "len", "response"])

for row in df_test_womens_tops_tunics.itertuples(index=False):
    image_id = str(row.id).zfill(6)
    image_path = f"/content/drive/MyDrive/MeeshoAIChallenge/test_images/{image_id}.jpg"
    print(image_path)

    if os.path.exists(image_path):
        raw_image = Image.open(image_path).convert('RGB')
        predicted_attributes = generate_combined_question(raw_image, sub_attr_womens_tops_tunics)

        with open(csv_file_path, mode='a', newline='') as file:
            writer = csv.writer(file, quoting=csv.QUOTE_MINIMAL)
            writer.writerow([image_id, category, length, predicted_attributes])
    else:
        print(f"Image {image_id}.jpg not found.")

/content/drive/MyDrive/MeeshoAIChallenge/test_images/023564.jpg
/content/drive/MyDrive/MeeshoAIChallenge/test_images/023565.jpg
/content/drive/MyDrive/MeeshoAIChallenge/test_images/023566.jpg
/content/drive/MyDrive/MeeshoAIChallenge/test_images/023567.jpg


KeyboardInterrupt: 